# Our Model - Building Complex of 4 Buildings with 5GHz

## Importing necessary tools

In [ ]:
!pip install ortools
from __future__ import print_function
from ortools.sat.python import cp_model as cp
import math 

#for visualisation
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Definition of model parameters

In [ ]:
model = cp.CpModel()
min_distance = 30 #reach of wavelengths

#number of rooms
s=12 #number of sections on one length (x or y axis) -> we have three apartments, but each apartment is divided into 4 sections/rooms (2 per axis)
f=5 #number of floors (rooms on z axis)
num_blocks = s*s*f   # number of total rooms to be covered

#dimensions or distance between points/walls
l=4  #lenght of room (can be changed) -> aparment length is 8m
w=4  #width of room (can be changed) -> apartment length is 8m
h=3  #height of room (always the same (3m))

#reach distance decrease because of walls
sw = 1.8   #small wall (room walls within apartments)
mw = 2.6   #medium walls (apartment walls between apartments)
bw = 5     #big walls (outside building walls)
c = 6.6    #ceiling
hd = 10    #distance between houses

## Distance matrix calculation with constraints of all wall sizes

In [ ]:
#calculating the distance between all rooms
def euclidean_distance(a, b):
  distance = math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))]))   #general equation to calculate distance between two points in 3 dimensions
  diffz = abs(a[0] - b[0])   #distance in z direction
  diffx = abs(a[1] - b[1])   #distance in x direction
  diffy = abs(a[2] - b[2])   #distance in y direction

    #ceiling effect
  if diffz > 0:
    distance += (diffz/h)*c     #add distance for every ceiling/floor that we pass through
    
    #small wall effect
  if diffx > 0:
    distance += (diffx/l)*sw    #add distance for every small wall that we pass through
  if diffy > 0:
    distance += (diffy/w)*sw    #add distance for every small wall that we pass through
    
    #medium wall effect
  if (diffx/l) % 2 == 0:                 #if the departments are a multiple of 2 apart from each other
    distance += ((diffx/l)//2) * mw     #then add how many multiples of 2 the department has times the thickness of the apartment wall
  else:
    if (a[1]<b[1] and (a[1]/l)%2==0) or (b[1]<a[1] and (b[1]/l)%2==0):  #if departments are uneven number apart and the lower indicated apartment is a multiple of 2
        distance += ((diffx/l)//2) * (mw-sw)                      #add multiple of 2 medium walls and subtract thin walls instead
    else:
        distance += ((diffx/l)//2) * (mw-sw) + (mw-sw)            #otherwise add the same plus one additional medium wall

  if (diffy/w) % 2 == 0:                 #if the departments are a multiple of 2 apart from each other
    distance += ((diffy/w)//2) * mw     #then add how many multiples of 2 the department has times the thickness of the apartment wall
  else:
    if (a[2]<b[2] and (a[2]/w)%2==0) or (b[2]<a[2] and (b[2]/w)%2==0):  #if departments are uneven number apart and the lower indicated apartment is a multiple of 2
        distance += ((diffy/w)//2) * (mw-sw)                      #add multiple of 2 medium walls and subtract thin walls instead
    else:
        distance += ((diffy/w)//2) * (mw-sw) + (mw-sw)            #otherwise add the same plus one additional medium wall
 
    #big wall effect
  if (a[1]/l < 6 and b[1]/l >= 6) or (a[1]/l > 6 and b[1]/l <= 6):   #if both points are on one half of the spectrum (above or below 6)
    distance += 2*bw + hd - sw - mw                               #add the two big outside walls and the distance between the buildings and subtract a small and medium wall
    
  if (a[2]/w < 6 and b[2]/w >= 6) or (a[2]/w > 6 and b[2]/w <= 6):   #if both points are on one half of the spectrum (above or below 6)
    distance += 2*bw + hd - sw - mw                               #add the two big outside walls and the distance between the buildings and subtract a small and medium wall
    
  return distance

## Running the optimization model

In [ ]:
#create model of room location
points = [(i, j, k) for i in range(f) for j in range(s) for k in range(s)]     #simple point matrix with distance 1 between points
selected_points = [(i*h, j*l, k*w) for i, j, k in points if i in list(range(f)) and j in list(range(s)) and k in list(range(s))]   #adapted point matrix according to real distances

#create distance matrix between all rooms
distance = [
    [euclidean_distance(a, b) for b in selected_points] for a in selected_points
]


# define decision variables - coverage of router x=1 if covered
x = [model.NewIntVar(0, 1, "x[%i]" % i) for i in range(num_blocks)] #decision, where we put routers (1 if we use router in room, 0 if not)
z = model.NewIntVar(0, num_blocks, "z") #total number of routers used (want to minimize)

#objective to minimize: 
model.Add(z == sum(x))

#constraints
for i in range(num_blocks):
        model.Add(sum([x[j] for j in range(num_blocks) if distance[i][j] <= min_distance]) >= 1)


model.Minimize(z)
solver = cp.CpSolver()
status = solver.Solve(model)
if status == cp.OPTIMAL:
  print("z:", solver.Value(z))
  print("x:", [solver.Value(x[i]) for i in range(num_blocks)])
  print("NumConflicts:", solver.NumConflicts())
  print("NumBranches:", solver.NumBranches())
print (len(x))

720


## Visualisation of results

In [ ]:
print (selected_points)

In [ ]:
# Extract the coordinates of selected points and router placements
coordinates = np.array([list(point) for point in selected_points])
router_placements = np.array([solver.Value(x[i]) for i in range(num_blocks)])

# Separate the router placements into individual coordinates
router_coordinates = coordinates[router_placements == 1]
display(router_coordinates)
rotated_router_coordinates = np.copy(router_coordinates)
rotated_router_coordinates[:,0] = router_coordinates[:,2]
rotated_router_coordinates[:,1] = router_coordinates[:, 1]
rotated_router_coordinates[:,2] = router_coordinates[:, 0]
display(rotated_router_coordinates)
# Rotate the coordinates and adjust the position
rotated_coordinates = np.copy(coordinates)
rotated_coordinates[:, 0], rotated_coordinates[:, 1], rotated_coordinates[:, 2] = coordinates[:, 2], coordinates[:, 1], coordinates[:, 0]

# Plotting the 3D visualization
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


# Plot all room coordinates
ax.scatter(rotated_coordinates[:, 0],rotated_coordinates[:, 1],rotated_coordinates[:, 2], c='blue', marker='s', label='Room')

# Plot router coordinates
ax.scatter(rotated_router_coordinates[:, 0], rotated_router_coordinates[:, 1], rotated_router_coordinates[:, 2], c='red', marker='^', s=110, alpha=0.8, edgecolors='black', label='Routers')

# Set labels and legend
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title("Wifi Router Placement")
ax.legend()

plt.show()

In [ ]:
# Plotting the 3D visualization
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot router coordinates
ax.scatter(rotated_router_coordinates[:, 0], rotated_router_coordinates[:, 1], rotated_router_coordinates[:, 2], c='red', marker='^', s=110, alpha=0.8, edgecolors='black', label='Routers')

# Set labels and legend
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title("Wifi Router Placement")
ax.legend()

plt.show()